## Conversational Interface - Chatbot with Claude-v2 and OpenSearch

### Amazon Bedrock을 사용하는 챗봇

In [1]:
installed_needed = False

In [2]:
import sys
import IPython

if installed_needed:
  print("installing deps and restarting kernel")
  !{sys.executable} -m pip install -U pip
  !{sys.executable} -m pip install -U sagemaker
  !{sys.executable} -m pip install "ipywidget>=7,<8"
  !{sys.executable} -m pip install langchain==0.0.347
  
  IPython.Application.instance().kernel.do_shutdown(True)

In [3]:
# add path

import sys, os
module_path=".."
sys.path.append(os.path.abspath(module_path))

### 1. Bedrock Client 생성

In [4]:
from utils import bedrock
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region="us-east-1"
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### 2. Titan Embedding 및 LLM Calud2-v2 모델 로딩

#### LLM loading (claude-v2)

In [5]:
from langchain.llms.bedrock import Bedrock

llm_text = Bedrock(
  client= boto3_bedrock,
  model_id="anthropic.claude-v2",
  model_kwargs={
    "max_tokens_to_sample": 512,
    "temperature": 0.1,
    "top_k": 3,
    "top_p": 0.1
  }
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f472764f310>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512, 'temperature': 0.1, 'top_k': 3, 'top_p': 0.1})

#### Embedding Model 선택

In [6]:
from langchain.embeddings.bedrock import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
  client=boto3_bedrock,
  model_id="amazon.titan-embed-text-v1",
)

### 3. LangChain OpenSearch VectorStore 생성
##### opensearch-py package를 이용하여 local opensearch 연결

ref: https://opensearch.org/docs/latest/clients/python-low-level/


#### Local OpenSearch 정보

In [7]:
host = "localhost"
port = "9200"
opensearch_endpoint = f"https://{host}:{port}"
# 인증 정보
http_auth = ("admin", "admin")

# CA에 대한 ssl 정보
ca_certs_path = "root-ca.pem"

# Optional Client certificates if you don't want to use HTTP basic authentication
client_cert_path = 'admin.pem'
client_key_path = 'admin-key.pem'

#### Create client with SSL/TLS, but hostname verification disabled

In [8]:
from opensearchpy import OpenSearch

os_client = OpenSearch(
  hosts=[{'host': host, 'port': port}],
  http_compress=True,
  http_auth=http_auth,
  client_cert = client_cert_path,
  client_key = client_key_path,
  use_ssl = True,
  verify_certs = True,
  ssl_assert_hostname = False,
  ssl_show_warn=False,
  ca_certs = ca_certs_path
 )

In [9]:
# index name
index_name = "genai-demo-index-v1-with-tokenizer"

#### LangChain OpenSearch VectorStore 생성

In [10]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
  index_name=index_name,
  opensearch_url=opensearch_endpoint,
  embedding_function=llm_emb,
  http_auth=http_auth,
  is_aoss=False,
  engine="faiss",
  space_type="l2",
  client_cert = client_cert_path,
  client_key = client_key_path,
  use_ssl = True,
  verify_certs = True,
  ssl_assert_hostname = False,
  ssl_show_warn=False,
  ca_certs = ca_certs_path
)

### 4. RetrievalQA 구현
##### 템플릿 생성

In [11]:
from langchain.prompts import PromptTemplate

prompt_template = """\n\nHuman: 다음 문맥의 information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요.

{context}

Question: {question}
\n\nAssistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

#### 필터 생성

In [12]:
from utils.rag import  run_RetrievalQA, show_context_used
from utils.opensearch import opensearch_utils

filter01 = "인터넷뱅킹"
filter02 = "신한은행"

query = "타기관 OTP 등록 방법 알려 주세요"

boolean_filter = opensearch_utils.get_filter(
  filter=[
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}}
  ]
)
print(boolean_filter)

{'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}}, {'term': {'metadata.source': '신한은행'}}]}}


In [13]:
from utils import print_ww
result = run_RetrievalQA(
  llm=llm_text,
  query=query,
  prompt=PROMPT,
  vector_db=vector_db,
  boolean_filter=boolean_filter,
  k=5
)

print("#"*50)
print("query: ", query)
print("#"*50)
print_ww(result['result'])

##################################################
query:  타기관 OTP 등록 방법 알려 주세요
##################################################
 타기관 OTP 등록 방법은 다음과 같습니다:

1. 인터넷뱅킹 로그인 → 사용자관리→인터넷뱅킹관리→OTP이용등록

2. 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록

※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금한 점은 신한은행 고객센터(1599-8000)로 문의하시기 바랍니다.


### 5. Conversational Chatbot 사용한 대화형 세션

### ConversationChain 생성

In [17]:
from langchain.chains import ConversationChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


In [44]:
from langchain.prompts.prompt import PromptTemplate

condense_question_template = """To create condense_question, given the conversation and a follow up question, rephrase the follo up question to be a standalone question,
in int original language

Chat History:
{chat_history}
Follow up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT =  PromptTemplate.from_template(condense_question_template)

qa_prompt = PromptTemplate.from_template(
"""
Human:
Here is the context, inside <context></context> XML tags.
Based on the context as below, answer the question. if you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

Human: Use at maximum 5 sentences to answer the following question.
{question}

If the answer is not in the context say "주어진 내용에서 관련 답변을 찾을 수 없습니다."

Assistant:"""
)

### 챗봇 (ConversationRetrievalChain) 질문

In [34]:
from utils.rag import ChatUX
from utils.rag import _get_chat_history

def create_ConversationalRetrievalChain(qa_prompt, bollean_filter, llm_text, k=1, verbose=False):
  memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
  qa = ConversationalRetrievalChain.from_llm(
    llm = llm_text,
    retriever=vector_db.as_retriever(
      search_type ="similarity",
      search_keyworkd= {
        "k": k,
        "boolean_filter": boolean_filter
      }
    ), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs={'prompt': qa_prompt},
    get_chat_history=_get_chat_history,
    verbose=verbose
  )

  return qa

### 6. 챗봇 (ConversationalRetrievalChain) 실행
#### ChatHistory 사용한 Q&A 내부로직
- 첫째 질문:
  - qa("타기관OPT 등록방법 알려 주세요")
  - 내부로직
    - (1) StuffDocumentChain 실행 (Vector Store로부터 검색하여 Top K가져옴)
    - (2) combine_docs_chain_kwargs(아래 정의 참조)를 이용하여 Prompt 포맷팅 완료 후에 LLMChain 실행
    - (3) 결과 제공

- 둘째 질문:
  - qa("앱 이름이 뭐야?")
  - 내부로직
    - (1) Chat History 존재 확인
    - (2) CONDENSE_QUESTION_PROMPT를 실행하여 새로이 입력 된 질문을 기존의 chat hsitory와 결합하여 새로운 질문을 생성함.
      - "새로운 질문" + "Chat History"= "새롭게 창조된 질문" (예: 타기관 OTP 등록 시 사용하는 앱 이름이 무엇인가요?
        - "타기관 OTP 등록 시 사용하는" 이 부분이 채팅 기록을 이용하여 만들어 냄
    - (3) StuffDocumentsChain 실행 ("새롭게 창조된 질문"을 이용하여 Vectorstore로 부터 검색하여 Top K를 가져 옴)
    - (4) combine_docs_chain_kwargs를 이용하여 prompt 포맷팅 완료 후에 LLMChain 실행
    - (5) 결과 제공

- CONDENSE_QUESTION_PROMPT
  - 우리가 제어할 수 있는 첫 번째 것은 채팅 기록과 새로운 질문을 가져와 독립형 질문을 생성하는 프롬프트입니다.
  이 독립형 질문이 관련 문서를 찾는 데 사용되기 때문에 필요합니다. 이는 중요한 단계입니다. 관련 문서를 조회하기 위해 새로운 질문을
  사용했다면, 후속 질문에 대해 챗봇이 제대로 작동하지 않을 것입니다.
- condense_docs_chain_kwargs={'prompt': qa_prompt}
  - 매번 대화를 할 때마다 생성이 되는 프롬프트입니다. 이 프롬프트는 claude 형식의 프롬프트로 만들어 주시길 권장합니다. 

#### (1) 챗봇 실행

In [45]:
qa = create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=3, verbose=True)

In [46]:
qa("타기관 OTP 등록 방법을 알려 주세요")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Human:
Here is the context, inside <context></context> XML tags.
Based on the context as below, answer the question. if you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지

/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human:
Here is the context, inside <context></context> XML tags.
Based on the context as below, answer the question. if you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인


> Finished chain.

> Finished chain.


{'question': '타기관 OTP 등록 방법을 알려 주세요',
 'chat_history': [HumanMessage(content='타기관 OTP 등록 방법을 알려 주세요'),
  AIMessage(content=' 주어진 내용에서는 타기관 OTP 등록 방법에 대한 정보가 포함되어 있지 않습니다.')],
 'answer': ' 주어진 내용에서는 타기관 OTP 등록 방법에 대한 정보가 포함되어 있지 않습니다.'}

In [37]:
qa("앱 이름이 뭐야")



> Entering new LLMChain chain...
Prompt after formatting:
To create condense_question, given the conversation and a follow up question, rephrase the follo up question to be a standalone question,
in int original language

Chat History:



Human: 타기관 OTP 등록 방법을 알려 주세요


Assistant:  타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 인터넷뱅킹 로그인후 사용자관리에서 OTP이용등록을 진행하시면 됩니다. OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.
Follow up Input: 앱 이름이 뭐야
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록

/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 


Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. 
푸시메세지 터치하여 모바일 OTP 비밀번호 6자리 입력, 인증처리 완료후 인터넷뱅킹에서 이후의 절차를 진행하시면 됩니다.
※ 스마트폰에 모바일 OTP 푸시(PUSH) 메세지가 오지 않는 경우 메뉴 직접 실행 
- 신한 쏠(SOL) [로그인 하지 않음]> 전체메뉴> 설정/인증> 보안매체 전체보기> 모바일OTP> 인증
기타 궁금하신 내용은 신한은행 고객센터 


> Finished chain.

> Finished chain.


{'question': '앱 이름이 뭐야',
 'chat_history': [HumanMessage(content='타기관 OTP 등록 방법을 알려 주세요'),
  AIMessage(content=' 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 인터넷뱅킹 로그인후 사용자관리에서 OTP이용등록을 진행하시면 됩니다. OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.'),
  HumanMessage(content='앱 이름이 뭐야'),
  AIMessage(content=" Sorry, I don't know as the answer was not found in the context.")],
 'answer': " Sorry, I don't know as the answer was not found in the context."}

In [38]:
qa("등록 후  재로그인하면 새로 등록한 OTP를 사용할 수 있어")



> Entering new LLMChain chain...
Prompt after formatting:
To create condense_question, given the conversation and a follow up question, rephrase the follo up question to be a standalone question,
in int original language

Chat History:



Human: 타기관 OTP 등록 방법을 알려 주세요


Assistant:  타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 인터넷뱅킹 로그인후 사용자관리에서 OTP이용등록을 진행하시면 됩니다. OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.


Human: 앱 이름이 뭐야


Assistant:  Sorry, I don't know as the answer was not found in the context.
Follow up Input: 등록 후  재로그인하면 새로 등록한 OTP를 사용할 수 있어
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 

/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 


Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. 
푸시메세지 터치하여 모바일 OTP 비밀번호 6자리 입력, 인증처리 완료후 인터넷뱅킹에서 이후의 절차를 진행하시면 됩니다.
※ 스마트폰에 모바일 OTP 푸시(PUSH) 메세지가 오지 않는 경우 메뉴 직접 실행 
- 신한 쏠(SOL) [로그인 하지 않음]> 전체메뉴> 설정/인증> 보안매체 전체보기> 모바일OTP> 인증
기타 궁금하신 내용은 신한은행 고객센터 


> Finished chain.

> Finished chain.


{'question': '등록 후  재로그인하면 새로 등록한 OTP를 사용할 수 있어',
 'chat_history': [HumanMessage(content='타기관 OTP 등록 방법을 알려 주세요'),
  AIMessage(content=' 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 인터넷뱅킹 로그인후 사용자관리에서 OTP이용등록을 진행하시면 됩니다. OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.'),
  HumanMessage(content='앱 이름이 뭐야'),
  AIMessage(content=" Sorry, I don't know as the answer was not found in the context."),
  HumanMessage(content='등록 후  재로그인하면 새로 등록한 OTP를 사용할 수 있어'),
  AIMessage(content=' 타기관에서 발급받은 OTP를 아마존 은행에 등록한 후 재로그인하면 새로 등록한 OTP를 사용할 수 있습니다.')],
 'answer': ' 타기관에서 발급받은 OTP를 아마존 은행에 등록한 후 재로그인하면 새로 등록한 OTP를 사용할 수 있습니다.'}

In [39]:
qa_d = create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=3, verbose=True)

In [40]:
qa_d("타기관OTP 등록 방법 알려주세요")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림 > 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 아마존은행 쏠(SOL) 로그인→ 전체

/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 


Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림 > 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에


> Finished chain.

> Finished chain.


{'question': '타기관OTP 등록 방법 알려주세요',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요'),
  AIMessage(content=' 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록 또는 모바일뱅킹 쏠(SOL)로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록을 통해 타기관 OTP를 등록할 수 있습니다.')],
 'answer': ' 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록 또는 모바일뱅킹 쏠(SOL)로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록을 통해 타기관 OTP를 등록할 수 있습니다.'}

In [41]:
qa_d("앱 이름이 뭐야?")



> Entering new LLMChain chain...
Prompt after formatting:
To create condense_question, given the conversation and a follow up question, rephrase the follo up question to be a standalone question,
in int original language

Chat History:



Human: 타기관OTP 등록 방법 알려주세요


Assistant:  인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록 또는 모바일뱅킹 쏠(SOL)로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록을 통해 타기관 OTP를 등록할 수 있습니다.
Follow up Input: 앱 이름이 뭐야?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등

/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 


Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. 
푸시메세지 터치하여 모바일 OTP 비밀번호 6자리 입력, 인증처리 완료후 인터넷뱅킹에서 이후의 절차를 진행하시면 됩니다.
※ 스마트폰에 모바일 OTP 푸시(PUSH) 메세지가 오지 않는 경우 메뉴 직접 실행 
- 신한 쏠(SOL) [로그인 하지 않음]> 전체메뉴> 설정/인증> 보안매체 전체보기> 모바일OTP> 인증
기타 궁금하신 내용은 신한은행 고객센터 


> Finished chain.

> Finished chain.


{'question': '앱 이름이 뭐야?',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요'),
  AIMessage(content=' 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록 또는 모바일뱅킹 쏠(SOL)로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록을 통해 타기관 OTP를 등록할 수 있습니다.'),
  HumanMessage(content='앱 이름이 뭐야?'),
  AIMessage(content=" Sorry, I don't know as the answer was not found in the context.")],
 'answer': " Sorry, I don't know as the answer was not found in the context."}

In [42]:
qa_d("등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?")



> Entering new LLMChain chain...
Prompt after formatting:
To create condense_question, given the conversation and a follow up question, rephrase the follo up question to be a standalone question,
in int original language

Chat History:



Human: 타기관OTP 등록 방법 알려주세요


Assistant:  인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록 또는 모바일뱅킹 쏠(SOL)로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록을 통해 타기관 OTP를 등록할 수 있습니다.


Human: 앱 이름이 뭐야?


Assistant:  Sorry, I don't know as the answer was not found in the context.
Follow up Input: 등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[

/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 


Human:
Here is the context, inside <context></context> XML tags.    
Based on the context as below, answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
 
 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림 > 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.

ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버


> Finished chain.

> Finished chain.


{'question': '등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?',
 'chat_history': [HumanMessage(content='타기관OTP 등록 방법 알려주세요'),
  AIMessage(content=' 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록 또는 모바일뱅킹 쏠(SOL)로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→ OTP이용등록을 통해 타기관 OTP를 등록할 수 있습니다.'),
  HumanMessage(content='앱 이름이 뭐야?'),
  AIMessage(content=" Sorry, I don't know as the answer was not found in the context."),
  HumanMessage(content='등록 후 재로그인하면 새로 등록한 OTP를 사용할 수 있어?'),
  AIMessage(content=' 네, 맞습니다. OTP를 등록한 후 재로그인하면 새로 등록한 OTP를 사용할 수 있습니다.')],
 'answer': ' 네, 맞습니다. OTP를 등록한 후 재로그인하면 새로 등록한 OTP를 사용할 수 있습니다.'}

In [43]:
qa_1 = create_ConversationalRetrievalChain(qa_prompt, boolean_filter, llm_text, k=3, verbose=False)

chat = ChatUX(qa_1, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()